# Imports

In [ ]:
import os
import re
import sys
import glob
import json
import numpy as np
import pandas as pd
from pathlib import Path
from itertools import product
from collections import OrderedDict


# Custom functions

## IMGT HLA sequence to dictionary

In [ ]:
def parse_imgt_four_digit(file_paths):
    """
    Parses IMGT/HLA protein FASTA files to extract unique HLA alleles at a 
    four-digit resolution (e.g., "A*01:01") for each allele found in the files. 

    Parameters:
        file_paths (list): List of paths to IMGT/HLA FASTA files.

    Returns:
        dict: A dictionary where keys are four-digit HLA allele identifiers, and values are 
              the protein sequences associated with each allele.
    """
    fasta_dict = {}
    
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            header = None
            sequence = []
            for line in file:
                line = line.strip()
                if line.startswith('>'):
                    if header and header not in fasta_dict:
                        # Save the previous sequence if it's the first occurrence
                        fasta_dict[header] = ''.join(sequence)
                    # The second part of the header (after the space) is the full HLA ID
                    hla_id_full = line.split()[1]  # "A*01:01:01:06"
                    
                    # Extract the four-digit resolution (e.g., "A*01:01")
                    hla_id_four_digit = ":".join(hla_id_full.split(":")[:2])  # "A*01:01"
                    
                    # Only proceed if this four-digit HLA ID hasn't been seen
                    if hla_id_four_digit not in fasta_dict:
                        header = hla_id_four_digit  # Use the four-digit resolution as the key
                    else:
                        header = None  # Ignore sequences that have been seen
                    sequence = []  # Reset the sequence list for the new record
                else:
                    sequence.append(line)

            # Save the last sequence in the file, if not already stored
            if header and header not in fasta_dict:
                fasta_dict[header] = ''.join(sequence)

    return fasta_dict

# Usage example: using glob to get all .fasta files in a directory
fasta_file_paths = glob.glob('data/tcr_llm/databases/IMGTHLA/fasta/*_prot.fasta')
combined_fasta_dict = OrderedDict(parse_imgt_four_digit(fasta_file_paths))

## Transform HLA information to 2-digit resolution

In [ ]:
def transform_mhc_restriction(value, fasta_dict):
    """
    Transforms HLA allele identifiers into a standardized format using a
    provided FASTA dictionary for four-digit resolution matching when
    applicable. This function covers both Class I and Class II HLA alleles,
    handling a variety of formats and applying default transformations when
    precise matches are unavailable.

    Args:
        value (str): An HLA allele identifier in various formats (e.g., 
            "HLA-B*07:02", "HLA-A2", "HLA-DPB1").
        fasta_dict (dict): A dictionary where keys are four-digit HLA allele
        identifiers and values are amino acid sequences.

    Returns:
        str: The transformed HLA allele identifier in standardized format
        (e.g., "B*07:02", "A*02:01"). If no transformation applies, the original
        value is returned. For unrecognized formats or a TypeError,
        returns pd.NA.

    Transformation Rules:
        - Class I (A, B, C):
            * Four-digit alleles (e.g., "HLA-B*07:02") -> Remove "HLA-" prefix.
            * Two-digit alleles (e.g., "HLA-A2") -> Convert to four-digit where
                possible by checking fasta_dict (e.g., "A*02:01" if "A*02" 
                matches in fasta_dict), otherwise defaults to ":01".
        
        - Class II (DPA, DPB, DQA, DQB, DRA, DRB):
            * Four-digit alleles (e.g., "HLA-DPB1*02:01") -> Remove "HLA-"
                prefix.
            * DRB with single field (e.g., "HLA-DRB1*03") -> Completes to
                "DRB1*03:01" if missing the second field.
            * No resolution (e.g., "HLA-DPB1") -> Adds "*01:01" if allele format
                matches a Class II allele.

        - Class I (HLA-E): Returns four-digit format by removing "HLA-" prefix
            (e.g., "HLA-E*01:03" -> "E*01:03").

    Example:
        transformed_value = transform_mhc_restriction("HLA-A2", fasta_dict)
        # Output: "A*02:01" (if "A*02" is found in fasta_dict) or "A*02:01" by
        # default.
    """
    # Class I: A, B, C alleles with typical formats like HLA-B*07:02
    try:
        if re.match(r'^HLA-[ABC]\*\d+:\d+$', value):  # Match HLA-B*07:02, etc.
            return value.replace('HLA-', '')  # Remove 'HLA-' prefix
        
        # Class I: A, B, C with 2-digit resolution (e.g., HLA-A2, HLA-B2)
        elif re.match(r'^HLA-[ABC](\d{1,2})$', value):  # Match HLA-A2, HLA-B02
            two_digit_allele = re.sub(r'HLA-([ABC])(\d{1,2})$', r'\1*\2', value)  # Convert to A*02, B*07
            # Ensure the number part is always 2 digits (e.g., A2 -> A*02)
            two_digit_allele = re.sub(r'([ABC])\*(\d{1})$', r'\1*0\2', two_digit_allele)  # Convert A*2 -> A*02
            
            # Look for the first 4-digit match in the sorted dictionary
            for key in fasta_dict:
                if key.startswith(two_digit_allele):
                    return key  # Return the first matching 4-digit allele
            return two_digit_allele + ":01"  # Default to A*02:01 or B*02:01 if no match found
        
        # Class II: DPA, DPB, DQA, DQB alleles with typical formats like HLA-DPB1*02:01
        elif re.match(r'^HLA-D[PQ][AB][12]?\*\d+:\d+$', value):  # Match HLA-DPB1*02:01, HLA-DQA1*05:01
            return value.replace('HLA-', '')  # Remove 'HLA-' prefix
        
        # Class II: DRB (handling DRB1, DRB3, DRB4, DRB5, DRB345)
        elif re.match(r'^HLA-DR[AB][1345]?\*\d+:\d+$', value):  # Match HLA-DRB1*15:01, HLA-DRB4*01:03, etc.
            return value.replace('HLA-', '')  # Remove 'HLA-' prefix
        elif re.match(r'^HLA-DR[AB][1345]?\*\d+$', value):  # Match HLA-DRB1*03 (one field)
            return re.sub(r'HLA-(DR[AB][1345]?)\*(\d+)$', r'\1*\2:01', value)  # Add missing second field

        
        # Class II: DPB1 with no resolution digits (e.g. HLA-DPB1)
        elif re.match(r'^HLA-D[PQ][AB][12]?$', value):  # Match HLA-DPB1, HLA-DPA1, etc.
            return re.sub(r'HLA-(D[PQ][AB][12]?)$', r'\1*01:01', value)  # Add missing fields, DPB1*01:01
        
        # Class I: HLA-E alleles (e.g., HLA-E*01:03)
        elif re.match(r'^HLA-E\*\d+:\d+$', value):  # Match HLA-E*01:03
            return value.replace('HLA-', '')  # Remove 'HLA-' prefix

        else:
            return value  # Return as is for other formats
    except TypeError:
        return(pd.NA)

# CEDAR
From the CEDAR database we queried for all human T-cell and MHC assays with positive as well as negative data. The query generated three different views that will parsed in the following sections. 
1. MHC-ligand table
2. Receptor table
3. T-cell table

## T-cell table
This view contain information of all T-cell reactivity assay against peptide and neo-antigens

In [ ]:
# Read the T-cell assay table from CEDAR
cedar_tcell_path = Path('data/tcr_llm/databases/CEDAR/cedar_tcell_assay_results_table_102824.tsv')
cedar_tcell_table = pd.read_csv(cedar_tcell_path, 
    sep="\t",
    dtype = {"Reference - PMID": "str"},
    names = ['study_id', 'epitope_type', 'peptide', # Rename columns
        'epitope_reference_name', 'epitope_source_molecule', 
        'epitope_source_organism', 'epitope_source_species', 'host_organism', 
        'host_population', 'host_sex', 'host_age', 'host_mhc_profile',
        'assay_method', 'assay_response', 'assay_outcome', 'assay_subject_count',
        'assay_positive_count', 'source_tissue',
        'mhc_restriction'], header=0).dropna(subset=['study_id']) # Remove any row with missing PMID
cedar_tcell_table['study_id'] = cedar_tcell_table['study_id'].astype('int64') # Store PMID as an interger
cedar_tcell_table = cedar_tcell_table.loc[cedar_tcell_table['assay_outcome'] == "Positive"] # Only keep results where the results were positive

### T-cell metadata table
The metadata aims to standardize data from across multiple source. All metadata table will contain the same columns and can be merged across datasets. The metadata table will indicate the source database, the study id for each entry and study ID type.

In [ ]:
cedar_tcell_metadata = cedar_tcell_table
cedar_tcell_metadata['data_source'] = 'cedar_tcell_assay'
cedar_tcell_metadata['study_id_type'] = 'PMID'
cedar_tcell_metadata = cedar_tcell_metadata[['data_source', 'study_id', 
    'study_id_type', 'host_organism', 'host_population', 'host_age', 'host_sex',
    'host_mhc_profile', 'source_tissue', 'epitope_type', 'peptide',
    'epitope_reference_name', 'epitope_source_molecule',
    'epitope_source_organism', 'mhc_restriction', 'assay_method',
    'assay_response', 'assay_outcome', 'assay_subject_count',
    'assay_positive_count']]
cedar_tcell_metadata

### T-cell sequence table
Sequence table aim to standardize the sequence data from all data sources. Each sequence table will contain the TRA and TRB CDR3 sequences, Peptide, MHC chain one and MHC chain two

In [ ]:
cedar_tcell_sequence_table = cedar_tcell_table[['peptide', 'mhc_restriction']]
# Apply the transformation function to the 'mhc_restriction' column
cedar_tcell_sequence_table['mhc_restriction'] = cedar_tcell_sequence_table['mhc_restriction'].apply(transform_mhc_restriction, fasta_dict=combined_fasta_dict)
cedar_tcell_sequence_table['mhc'] = cedar_tcell_sequence_table['mhc_restriction'].apply(lambda mhc: combined_fasta_dict.get(mhc, pd.NA))
cedar_tcell_sequence_table['peptide'] = cedar_tcell_sequence_table['peptide'].str.split('+').str[0].str.strip()
cedar_tcell_sequence_table['tra'] = pd.NA
cedar_tcell_sequence_table['trb'] = pd.NA
cedar_tcell_sequence_table['sequence'] =  cedar_tcell_sequence_table['peptide'] + ' ' + cedar_tcell_sequence_table['mhc'] + ';'
cedar_tcell_sequence_table = cedar_tcell_sequence_table.drop(columns=['mhc_restriction'])
cedar_tcell_sequence_table = cedar_tcell_sequence_table[['tra', 'trb', 'peptide', 'mhc', 'sequence']]
cedar_tcell_sequence_table

## MHC ligand table
MHC ligand assay table catalogs all the positive peptide-MHC interaction reported in CEDAR

In [ ]:
# MHC ligand table
cedar_mhc_ligand_path = Path('data/tcr_llm/databases/CEDAR/cedar_mhc_ligand_assay_results_table_102824.tsv')
cedar_mhc_ligand_table = pd.read_csv(cedar_mhc_ligand_path, sep="\t",
    dtype = {"Reference - PMID": "str"},
    names = ['study_id', 'epitope_type', 'peptide',
        'epitope_reference_name', 'epitope_source_molecule', 'epitope_source_organism',
        'epitope_source_species', 'host_organism', 'host_population',
        'host_sex', 'host_age', 'host_mhc_profile',
        'assay_method', 'assay_response', 'assay_outcome', 'assay_subject_count',
        'assay_positive_count', 'source_tissue',
        'mhc_restriction'], header=0).dropna(subset=['study_id'])
cedar_mhc_ligand_table['study_id'] = cedar_mhc_ligand_table['study_id'].astype('int64')
# Retain just the positive peptide-MHC interactions
cedar_mhc_ligand_table = cedar_mhc_ligand_table.loc[cedar_mhc_ligand_table['assay_outcome'] == "Positive"]


### MHC ligand metadata table

In [ ]:
cedar_mhc_ligand_metadata = cedar_mhc_ligand_table
cedar_mhc_ligand_metadata['data_source'] = 'cedar_tcell_assay'
cedar_mhc_ligand_metadata['study_id_type'] = 'PMID'
cedar_mhc_ligand_metadata = cedar_mhc_ligand_metadata[['data_source',
    'study_id', 'study_id_type', 'host_organism', 'host_population',
    'host_age', 'host_sex', 'host_mhc_profile', 'source_tissue', 
    'epitope_type', 'peptide', 'epitope_reference_name',
    'epitope_source_molecule', 'epitope_source_organism', 'mhc_restriction',
    'assay_method', 'assay_response', 'assay_outcome', 'assay_subject_count',
    'assay_positive_count']]
cedar_mhc_ligand_metadata

### MHC ligand sequence table

In [ ]:
cedar_mhc_ligand_sequence_table = cedar_mhc_ligand_table[['peptide', 'mhc_restriction']]
# Apply the transformation function to the 'mhc_restriction' column
cedar_mhc_ligand_sequence_table['mhc_restriction'] = cedar_mhc_ligand_sequence_table['mhc_restriction'].apply(transform_mhc_restriction, fasta_dict=combined_fasta_dict)
cedar_mhc_ligand_sequence_table['mhc'] = cedar_mhc_ligand_sequence_table['mhc_restriction'].apply(lambda mhc: combined_fasta_dict.get(mhc, pd.NA))
cedar_mhc_ligand_sequence_table['peptide'] = cedar_mhc_ligand_sequence_table['peptide'].str.split('+').str[0].str.strip()
cedar_mhc_ligand_sequence_table['tra'] = pd.NA
cedar_mhc_ligand_sequence_table['trb'] = pd.NA
cedar_mhc_ligand_sequence_table['sequence'] =  cedar_mhc_ligand_sequence_table['peptide'] + ' ' + cedar_mhc_ligand_sequence_table['mhc'] + ';'
cedar_mhc_ligand_sequence_table = cedar_mhc_ligand_sequence_table.drop(columns=['mhc_restriction'])
cedar_mhc_ligand_sequence_table = cedar_mhc_ligand_sequence_table[['tra', 'trb', 'peptide', 'mhc', 'sequence']]


In [ ]:
cedar_mhc_ligand_sequence_table

## Receptor table

In [ ]:
# Receptor table
cedar_receptor_path = Path('data/tcr_llm/databases/CEDAR/cedar_receptor_results_table_102924.tsv')
cedar_receptor_table = pd.read_csv(cedar_receptor_path, sep="\t",
    names = ['recepetor_reference_name', 'study_id', 'peptide',
        'epitope_source_molecule', 'epitope_source_organism',
        'assay_method', 'chain_one_type', 'trav_gene','trad_gene', 'traj_gene', 
        'tra_protein_sequence', 'tra_junction_aa', 'tra_cdr1', 'tra_cdr2', 
        'chain_two_type', 'trbv_gene', 'trbd_gene', 'trbj_gene',
        'trb_protein_sequence', 'trb_junction_aa', 'trb_cdr1',
        'trb_cdr2'], header=0, index_col=False)
cedar_receptor_table['study_id'] = cedar_receptor_table['study_id'].str.extract(r'(\d{7})')
cedar_receptor_table['study_id'] = 'CDID' + cedar_receptor_table['study_id']

### Receptor metadata table

In [ ]:
cedar_receptor_metadata = cedar_receptor_table
cedar_receptor_metadata['data_source'] = 'cedar_receptor_table'
cedar_receptor_metadata['study_id_type'] = 'CEDAR'
cedar_receptor_metadata = cedar_receptor_metadata[['data_source', 'study_id',
    'study_id_type', 'trav_gene', 'trad_gene', 'traj_gene',
    'tra_junction_aa', 'trbv_gene', 'trbd_gene', 'trbj_gene', 'trb_junction_aa',
    'epitope_type', 'peptide', 'epitope_source_molecule', 
    'epitope_source_organism']]
cedar_receptor_metadata


### Receptor sequence table

In [ ]:
# Pull out the tra, trb, peptide, and MHC sequences
cedar_receptor_sequence_table = cedar_receptor_table[['peptide', 'tra_junction_aa', 'trb_junction_aa']]
cedar_receptor_sequence_table = cedar_receptor_sequence_table.rename(columns={'tra_junction_aa': 'tra', 'trb_junction_aa': 'trb'})
cedar_receptor_sequence_table['peptide'] = cedar_receptor_sequence_table['peptide'].str.split('+').str[0].str.strip()
cedar_receptor_sequence_table = cedar_receptor_sequence_table.drop_duplicates()
cedar_receptor_sequence_table['sequence'] = cedar_receptor_sequence_table[['tra', 'trb', 'peptide']].agg(lambda x: ' '.join(x.dropna()) + ';', axis=1)
cedar_receptor_sequence_table = cedar_receptor_sequence_table[['tra', 'trb', 'peptide', 'sequence']]
cedar_receptor_sequence_table

## CEDAR Merged sequence table

In [ ]:
# Concat the three sequence tables from the three different CEDAR views
cedar_sequence_table = pd.concat([cedar_tcell_sequence_table, cedar_mhc_ligand_sequence_table, cedar_receptor_sequence_table])
cedar_sequence_table = cedar_sequence_table.drop_duplicates()
cedar_sequence_table

# IEDB
From the IEDB database we queried for all human T-cell and MHC assays with positive as well as negative data. The query generated three different views that will parsed in the following sections. 
1. MHC-ligand table
2. Receptor table
3. T-cell table

**NOTE**: IEDB and CEDAR follow a very similar structure. The notebook replicates the code, but the data loader scripts will have a single script to 
read and format IEDB and CEDAR

## T-cell assay table

In [ ]:
# T-cell table
iedb_tcell_path = Path('quest/data/tcr_llm/databases/IEDB/iedb_tcell_assay_results_table_102924.tsv')
iedb_tcell_table = pd.read_csv(iedb_tcell_path, 
    sep="\t",
    dtype = {"Reference - PMID": "str"},
    names = ['study_id', 'epitope_type', 'peptide',
        'epitope_reference_name', 'epitope_source_molecule', 'epitope_source_organism',
        'epitope_source_species', 'host_organism', 'host_population',
        'host_sex', 'host_age', 'host_mhc_profile',
        'assay_method', 'assay_response', 'assay_outcome', 'assay_subject_count',
        'assay_positive_count', 'source_tissue',
        'mhc_restriction'], header=0).dropna(subset=['study_id'])
iedb_tcell_table['study_id'] = iedb_tcell_table['study_id'].astype('int64')
iedb_tcell_table = iedb_tcell_table.loc[iedb_tcell_table['assay_outcome'] == "Positive"]

### T-cell assay metadata table

In [ ]:
iedb_tcell_metadata = iedb_tcell_table
iedb_tcell_metadata['data_source'] = 'iedb_tcell_assay'
iedb_tcell_metadata['study_id_type'] = 'PMID'
iedb_tcell_metadata = iedb_tcell_metadata[['data_source', 'study_id',
    'study_id_type', 'host_organism', 'host_population', 'host_age', 'host_sex',
    'host_mhc_profile', 'source_tissue', 'epitope_type', 'peptide',
    'epitope_reference_name', 'epitope_source_molecule',
    'epitope_source_organism', 'mhc_restriction', 'assay_method',
    'assay_response', 'assay_outcome', 'assay_subject_count', 
    'assay_positive_count']]
iedb_tcell_metadata

### T-cell assay sequence table

In [ ]:
iedb_tcell_sequence_table = iedb_tcell_table[['peptide', 'mhc_restriction']]
# Apply the transformation function to the 'mhc_restriction' column
iedb_tcell_sequence_table['mhc_restriction'] = iedb_tcell_sequence_table['mhc_restriction'].apply(transform_mhc_restriction, fasta_dict=combined_fasta_dict)
iedb_tcell_sequence_table['mhc'] = iedb_tcell_sequence_table['mhc_restriction'].apply(lambda mhc: combined_fasta_dict.get(mhc, pd.NA))
iedb_tcell_sequence_table['peptide'] = iedb_tcell_sequence_table['peptide'].str.split('+').str[0].str.strip()
iedb_tcell_sequence_table['tra'] = pd.NA
iedb_tcell_sequence_table['trb'] = pd.NA
iedb_tcell_sequence_table['sequence'] =  iedb_tcell_sequence_table['peptide'] + ' ' + iedb_tcell_sequence_table['mhc'] + ';'
iedb_tcell_sequence_table = iedb_tcell_sequence_table.drop(columns=['mhc_restriction']).drop_duplicates()
iedb_tcell_sequence_table

## MHC Ligand assay table

### MHC ligand metadata table

In [ ]:
# MHC ligand table
iedb_mhc_ligand_path = Path('data/tcr_llm/databases/IEDB/iedb_mhcligand_assay_results_table_103024.tsv')
iedb_mhc_ligand_table = pd.read_csv(iedb_mhc_ligand_path, sep="\t",
    dtype = {"Reference - PMID": "str"},
    names = ['study_id', 'epitope_type', 'peptide',
        'epitope_reference_name', 'epitope_source_molecule',
        'epitope_source_organism', 'epitope_source_species', 'host_organism', 
        'host_population', 'host_sex', 'host_age', 'host_mhc_profile',
        'assay_method', 'assay_response', 'assay_outcome', 
        'assay_subject_count', 'assay_positive_count', 'source_tissue',
        'mhc_restriction'], header=0).dropna(subset=['study_id'])
iedb_mhc_ligand_table['study_id'] = iedb_mhc_ligand_table['study_id'].astype('int64')
iedb_mhc_ligand_table = iedb_mhc_ligand_table.loc[iedb_mhc_ligand_table['assay_outcome'] == "Positive"]
iedb_mhc_ligand_metadata = iedb_mhc_ligand_table
iedb_mhc_ligand_metadata['data_source'] = 'iedb_mhc_ligand_assay'
iedb_mhc_ligand_metadata['study_id_type'] = 'PMID'
iedb_mhc_ligand_metadata = iedb_mhc_ligand_metadata[['data_source', 'study_id',
    'study_id_type', 'host_organism', 'host_population',
    'host_age', 'host_sex', 'host_mhc_profile', 'source_tissue', 
    'epitope_type', 'peptide', 'epitope_reference_name',
    'epitope_source_molecule', 'epitope_source_organism', 'mhc_restriction',
    'assay_method', 'assay_response', 'assay_outcome', 'assay_subject_count',
    'assay_positive_count']]
iedb_mhc_ligand_metadata

### MHC ligand sequence table

In [ ]:
iedb_mhc_ligand_sequence_table = iedb_mhc_ligand_table[['peptide', 'mhc_restriction']]
# Apply the transformation function to the 'mhc_restriction' column
iedb_mhc_ligand_sequence_table['mhc_restriction'] = iedb_mhc_ligand_sequence_table['mhc_restriction'].apply(transform_mhc_restriction, fasta_dict=combined_fasta_dict)
iedb_mhc_ligand_sequence_table['mhc'] = iedb_mhc_ligand_sequence_table['mhc_restriction'].apply(lambda mhc: combined_fasta_dict.get(mhc, pd.NA))
iedb_mhc_ligand_sequence_table['peptide'] = iedb_mhc_ligand_sequence_table['peptide'].str.split('+').str[0].str.strip()
iedb_mhc_ligand_sequence_table['tra'] = pd.NA
iedb_mhc_ligand_sequence_table['trb'] = pd.NA
iedb_mhc_ligand_sequence_table = iedb_mhc_ligand_sequence_table.loc[~iedb_mhc_ligand_sequence_table['peptide'].str.contains(r'[a-z0-9]', regex=True)]
iedb_mhc_ligand_sequence_table['peptide'] = iedb_mhc_ligand_sequence_table['peptide'].str.replace(r'\s+', '', regex=True)
iedb_mhc_ligand_sequence_table['sequence'] =  iedb_mhc_ligand_sequence_table['peptide'] + ' ' + iedb_mhc_ligand_sequence_table['mhc'] + ';'
iedb_mhc_ligand_sequence_table = iedb_mhc_ligand_sequence_table.drop(columns=['mhc_restriction'])
iedb_mhc_ligand_sequence_table = iedb_mhc_ligand_sequence_table[['tra', 'trb', 'peptide', 'mhc', 'sequence']].drop_duplicates()

In [ ]:
iedb_mhc_ligand_sequence_table

## Receptor table

In [ ]:
# Receptor table
iedb_receptor_path = Path('data/tcr_llm/databases/IEDB/iedb_receptor_results_table_103024.tsv')
iedb_receptor_table = pd.read_csv(iedb_receptor_path, sep="\t",low_memory=False,
    names = ['study_id', 'recepetor_reference_name', 'receptor_type', 'peptide',
        'epitope_source_molecule', 'epitope_source_organism',
        'assay_method', 'mhc_restriction', 'chain_one_type', 'trav_gene','trad_gene', 'traj_gene', 
        'tra_protein_sequence', 'tra_junction_aa', 'tra_cdr1', 'tra_cdr2', 
        'chain_two_type', 'trbv_gene', 'trbd_gene', 'trbj_gene',
        'trb_protein_sequence', 'trb_junction_aa', 'trb_cdr1',
        'trb_cdr2'], header=0, index_col=False)
iedb_receptor_table['study_id'] = 'IEDBID' + iedb_receptor_table['study_id'].astype(str)
iedb_receptor_table = iedb_receptor_table.drop_duplicates()
iedb_receptor_table

### Receptor metadata table

In [ ]:
iedb_receptor_metadata = iedb_receptor_table
iedb_receptor_metadata['data_source'] = 'iedb_receptor_table'
iedb_receptor_metadata['study_id_type'] = 'IEDB'
iedb_receptor_metadata = iedb_receptor_metadata[['data_source', 'study_id', 
    'study_id_type', 'trav_gene', 'trad_gene', 'traj_gene', 'tra_junction_aa',
    'trbv_gene', 'trbd_gene', 'trbj_gene', 'trb_junction_aa', 'peptide', 
    'epitope_source_molecule', 'epitope_source_organism', 'mhc_restriction']]
iedb_receptor_metadata


### Receptor sequence table

In [ ]:
iedb_receptor_sequence_table = iedb_receptor_table[['tra_junction_aa', 'trb_junction_aa', 'peptide', 'mhc_restriction']]
iedb_receptor_sequence_table['mhc_restriction'] = iedb_receptor_sequence_table['mhc_restriction'].str.split(', ')
iedb_receptor_sequence_table = iedb_receptor_sequence_table.explode('mhc_restriction')
iedb_receptor_sequence_table['mhc_restriction'] = iedb_receptor_sequence_table['mhc_restriction'].apply(transform_mhc_restriction, fasta_dict=combined_fasta_dict)
iedb_receptor_sequence_table['mhc'] = iedb_receptor_sequence_table['mhc_restriction'].apply(lambda mhc: combined_fasta_dict.get(mhc, pd.NA))
iedb_receptor_sequence_table = iedb_receptor_sequence_table.rename(columns={'tra_junction_aa': 'tra', 'trb_junction_aa': 'trb'})
iedb_receptor_sequence_table = iedb_receptor_sequence_table.drop(columns=['mhc_restriction']).drop_duplicates()
iedb_receptor_sequence_table = iedb_receptor_sequence_table.loc[~iedb_receptor_sequence_table['peptide'].str.contains(r'[a-z0-9]', regex=True)]
iedb_receptor_sequence_table['peptide'] = iedb_receptor_sequence_table['peptide'].str.replace(r'\s+', '', regex=True)
iedb_receptor_sequence_table['peptide'] = iedb_receptor_sequence_table['peptide'].str.split('+').str[0].str.strip()
iedb_receptor_sequence_table = iedb_receptor_sequence_table.drop_duplicates()
iedb_receptor_sequence_table['sequence'] = iedb_receptor_sequence_table[['tra', 'trb', 'peptide', 'mhc']].agg(lambda x: ' '.join(x.dropna()) + ';', axis=1)
iedb_receptor_sequence_table = iedb_receptor_sequence_table[['tra', 'trb', 'peptide', 'mhc', 'sequence']]
iedb_receptor_sequence_table.loc[~iedb_receptor_sequence_table['tra'].isna() & ~iedb_receptor_sequence_table['trb'].isna()]

## IEDB Merged sequences

In [ ]:
iedb_sequence_table = pd.concat([iedb_tcell_sequence_table, iedb_mhc_ligand_sequence_table, iedb_receptor_sequence_table])
iedb_sequence_table = iedb_sequence_table.drop_duplicates()
iedb_sequence_table

# IMGT HLA

IMGT HLA maintains a database of all known and validated MHC alleles. Our training data will contain the sequences for all MHC allele reported in humans

In [ ]:
imgt_hla_sequence_table = pd.DataFrame(combined_fasta_dict.values(), columns=['mhc'])
imgt_hla_sequence_table['sequence'] = imgt_hla_sequence_table['mhc'] + ';'
imgt_hla_sequence_table

# iReceptor

iReceptor is a public database that store TCR sequencing datasets from publications. These are stored in six parts:
1. AIRR COVID 19
2. iReceptor public archive 1
3. iReceptor public archive 2
4. iReceptor public archive 3
5. University of munster
6. VDJServer

## AIRR COVID 19
This database contains 85130 paired TCR sequences (from https://covid19-1.ireceptor.org/airr/v1/). Primarily from the 10X platform. The metadata information for each sample is stored in a JSON file.

In [ ]:
airr_covid_path = Path('data/tcr_llm/databases/ireceptor/airr-covid-19.tsv')
airr_covid_table = pd.read_csv(airr_covid_path, sep = "\t")
airr_covid_table = airr_covid_table[['data_processing_id', 'repertoire_id', 
    'cell_id', 'clone_id', 'productive', 'locus', 'v_call', 'd_call', 'j_call',
    'cdr1_aa', 'cdr2_aa', 'cdr3_aa', 'junction_aa']]
airr_covid_table = airr_covid_table.loc[airr_covid_table['productive'] == 'T']
airr_covid_tra_table = airr_covid_table.loc[airr_covid_table['locus'] == 'TRA'][['data_processing_id', 'cell_id', 'clone_id', 'v_call', 'd_call', 'j_call', 'junction_aa']]
airr_covid_tra_table = airr_covid_tra_table.rename(columns={'v_call': 'trav_gene',
    'd_call': 'trad_gene', 'j_call': 'traj_gene', 'junction_aa': 'tra_junction_aa'})
airr_covid_trb_table = airr_covid_table.loc[airr_covid_table['locus'] == 'TRB'][['data_processing_id', 'cell_id', 'clone_id', 'v_call', 'd_call', 'j_call', 'junction_aa']]
airr_covid_trb_table = airr_covid_trb_table.rename(columns={'v_call': 'trbv_gene',
    'd_call': 'trbd_gene', 'j_call': 'trbj_gene', 'junction_aa': 'trb_junction_aa'})
airr_covid_tcr_table = pd.merge(airr_covid_tra_table, airr_covid_trb_table, 
    on=['data_processing_id', 'cell_id', 'clone_id']).rename(columns={'data_processing_id': 'repertoire_id'})
airr_covid_tcr_table = airr_covid_tcr_table.drop(columns=['cell_id', 'clone_id'])

In [ ]:
airr_covid_meta_path = Path("data/tcr_llm/databases/ireceptor/airr-covid-19-metadata.json")
with open(airr_covid_meta_path) as meta_file:
    airr_covid_metadata = json.load(meta_file)
airr_metadata_list = list()
for repertoires in airr_covid_metadata["Repertoire"]:
    repertoire_dict = dict()
    repertoire_dict['study_id'] = repertoires['study']['study_id']
    repertoire_dict['repertoire_id'] = repertoires['data_processing'][0]['data_processing_id']
    repertoire_dict['host_organism'] = repertoires['subject']['species']['label']
    if repertoires['subject']['diagnosis'][0]['disease_diagnosis']['label'] == None:
        repertoire_dict['condition_studies'] = repertoires['subject']['diagnosis'][0]['study_group_description']
    else:
        repertoire_dict['condition_studies'] = repertoires['subject']['diagnosis'][0]['study_group_description'] + ' ' + repertoires['subject']['diagnosis'][0]['disease_diagnosis']['label']
    repertoire_dict['age'] = repertoires['subject']['age']
    repertoire_dict['sex'] = repertoires['subject']['sex']
    repertoire_dict['population_surveyed'] = repertoires['subject']['race']
    mhc_list = set()
    for mhc_genotypes in repertoires['subject']['genotype']['mhc_genotype_set']['mhc_genotype_list']:
        for alleles in mhc_genotypes['mhc_alleles']:
            if alleles['allele_designation'] == None:
                continue
            else:
                mhc_list.add(alleles['allele_designation'])
    repertoire_dict['mhc_profile'] = ','.join(sorted(list(mhc_list))) if list(mhc_list) else pd.NA
    repertoire_dict['source_tissue'] = repertoires['sample'][0]['tissue']['label']
    airr_metadata_list.append(repertoire_dict)
airr_covid_metadata_table = pd.DataFrame(airr_metadata_list).drop_duplicates()
airr_covid_metadata_table = pd.merge(airr_covid_metadata_table, airr_covid_tcr_table, on = "repertoire_id", how = "inner")
airr_covid_metadata_table

In [ ]:
airr_covid_sequence_table = airr_covid_metadata_table[['tra_junction_aa', 'trb_junction_aa']].rename(columns={'tra_junction_aa': 'tra', 'trb_junction_aa': 'trb'})
airr_covid_sequence_table['sequence'] = airr_covid_sequence_table['tra'] + ' ' + airr_covid_sequence_table['trb'] + ';'
airr_covid_sequence_table = airr_covid_sequence_table.drop_duplicates()

## iReceptor public archive part 1

Contains bulk TCR sequencing data with 201960232 sequences (from https://ipa4.ireceptor.org/airr/v1/). Majority of sequences are TRB CDR3 sequences

In [ ]:
ireceptor_paone_path = Path('/home/sravisha/projects/quest/data/tcr_llm/databases/ireceptor/ireceptor-public-archive_part1.tsv')
ireceptor_paone_table = pd.read_csv(ireceptor_paone_path, sep = "\t", nrows=500000)
ireceptor_paone_table = ireceptor_paone_table[['repertoire_id', 
    'cell_id', 'clone_id', 'productive', 'locus', 'v_call', 'd_call', 'j_call',
    'cdr1_aa', 'cdr2_aa', 'cdr3_aa', 'junction_aa']]
ireceptor_paone_table = ireceptor_paone_table.loc[ireceptor_paone_table['productive'] == 'T']
ireceptor_paone_trb_table = ireceptor_paone_table.loc[ireceptor_paone_table['locus'] == 'TRB'][['repertoire_id',  'cell_id', 'clone_id', 'v_call', 'd_call', 'j_call', 'junction_aa']]
ireceptor_paone_trb_table['repertoire_id'] = ireceptor_paone_table['repertoire_id'].astype('str')

### iReceptor PA1 metadata table

In [ ]:
ireceptor_paone_meta_path = Path("/home/sravisha/projects/quest/data/tcr_llm/databases/ireceptor/ireceptor-public-archive_part1-metadata.json")
with open(ireceptor_paone_meta_path) as meta_file:
    ireceptor_paone_metadata = json.load(meta_file)
ireceptor_paone_metadata_list = list()
for repertoires in ireceptor_paone_metadata["Repertoire"]:
    repertoire_dict = dict()
    repertoire_dict['study_id'] = repertoires['study']['study_id']
    repertoire_dict['repertoire_id'] = repertoires['repertoire_id']
    repertoire_dict['host_organism'] = repertoires['subject']['species']['label']
    if repertoires['subject']['diagnosis'][0]['disease_diagnosis']['label'] == None:
        repertoire_dict['condition_studies'] = repertoires['subject']['diagnosis'][0]['study_group_description']
    else:
        repertoire_dict['condition_studies'] = repertoires['subject']['diagnosis'][0]['study_group_description'] + ' ' + repertoires['subject']['diagnosis'][0]['disease_diagnosis']['label']
    repertoire_dict['age'] = repertoires['subject']['age']
    repertoire_dict['sex'] = repertoires['subject']['sex']
    repertoire_dict['population_surveyed'] = repertoires['subject']['race']
    mhc_list = set()
    for mhc_genotypes in repertoires['subject']['genotype']['mhc_genotype_set']['mhc_genotype_list']:
        for alleles in mhc_genotypes['mhc_alleles']:
            if alleles['allele_designation'] == None:
                continue
            else:
                mhc_list.add(alleles['allele_designation'])
    repertoire_dict['mhc_profile'] = ','.join(sorted(list(mhc_list))) if list(mhc_list) else pd.NA
    repertoire_dict['source_tissue'] = repertoires['sample'][0]['tissue']['label']
    ireceptor_paone_metadata_list.append(repertoire_dict)
ireceptor_paone_metadata_table = pd.DataFrame(ireceptor_paone_metadata_list).drop_duplicates()
ireceptor_paone_metadata_table['repertoire_id'] = ireceptor_paone_metadata_table['repertoire_id'].astype('str')
ireceptor_paone_metadata_table = pd.merge(ireceptor_paone_metadata_table, ireceptor_paone_trb_table, on = "repertoire_id", how = "inner")
ireceptor_paone_metadata_table

### iReceptor PA1 sequence table

In [ ]:
ireceptor_paone_sequence_table = ireceptor_paone_metadata_table[['junction_aa']]
ireceptor_paone_sequence_table = ireceptor_paone_sequence_table.loc[ireceptor_paone_sequence_table['junction_aa'].notna()]
ireceptor_paone_sequence_table = ireceptor_paone_sequence_table.loc[~ireceptor_paone_sequence_table['junction_aa'].str.contains('_')]
ireceptor_paone_sequence_table = ireceptor_paone_sequence_table.rename(columns={'junction_aa': 'trb'})
ireceptor_paone_sequence_table['sequence'] = ireceptor_paone_sequence_table['trb'] + ';'
ireceptor_paone_sequence_table = ireceptor_paone_sequence_table.drop_duplicates()
ireceptor_paone_sequence_table

## iReceptor public archive 2

This second part of the iReceptor database pre-dominantly contains bulkd TRB sequencing data with 156523762 sequences (from https://ipa5.ireceptor.org/airr/v1/)

In [ ]:
ireceptor_patwo_path = Path("data/tcr_llm/databases/ireceptor/ireceptor-public-archive_part2.tsv')
ireceptor_patwo_table = pd.read_csv(ireceptor_patwo_path, sep = "\t", nrows=500000)
ireceptor_patwo_table = ireceptor_patwo_table[['repertoire_id', 
    'cell_id', 'clone_id', 'productive', 'locus', 'v_call', 'd_call', 'j_call',
    'cdr1_aa', 'cdr2_aa', 'cdr3_aa', 'junction_aa']]
ireceptor_patwo_table = ireceptor_patwo_table.loc[ireceptor_patwo_table['productive'] == 'T']
ireceptor_patwo_trb_table = ireceptor_patwo_table.loc[ireceptor_patwo_table['locus'] == 'TRB'][['repertoire_id',  'cell_id', 'clone_id', 'v_call', 'd_call', 'j_call', 'junction_aa']]
ireceptor_patwo_trb_table['repertoire_id'] = ireceptor_patwo_table['repertoire_id'].astype('str')

### iReceptor PA2 metadata

In [ ]:
ireceptor_patwo_meta_path = Path("data/tcr_llm/databases/ireceptor/ireceptor-public-archive_part2-metadata.json")
with open(ireceptor_patwo_meta_path) as meta_file:
    ireceptor_patwo_metadata = json.load(meta_file)
ireceptor_patwo_metadata_list = list()
for repertoires in ireceptor_patwo_metadata["Repertoire"]:
    repertoire_dict = dict()
    repertoire_dict['study_id'] = repertoires['study']['study_id']
    repertoire_dict['repertoire_id'] = repertoires['repertoire_id']
    repertoire_dict['host_organism'] = repertoires['subject']['species']['label']
    if repertoires['subject']['diagnosis'][0]['disease_diagnosis']['label'] == None:
        repertoire_dict['condition_studies'] = repertoires['subject']['diagnosis'][0]['study_group_description']
    else:
        repertoire_dict['condition_studies'] = repertoires['subject']['diagnosis'][0]['study_group_description'] + ' ' + repertoires['subject']['diagnosis'][0]['disease_diagnosis']['label']
    repertoire_dict['age'] = repertoires['subject']['age']
    repertoire_dict['sex'] = repertoires['subject']['sex']
    repertoire_dict['population_surveyed'] = repertoires['subject']['race']
    mhc_list = set()
    for mhc_genotypes in repertoires['subject']['genotype']['mhc_genotype_set']['mhc_genotype_list']:
        for alleles in mhc_genotypes['mhc_alleles']:
            if alleles['allele_designation'] == None:
                continue
            else:
                mhc_list.add(alleles['allele_designation'])
    repertoire_dict['mhc_profile'] = ','.join(sorted(list(mhc_list))) if list(mhc_list) else pd.NA
    repertoire_dict['source_tissue'] = repertoires['sample'][0]['tissue']['label']
    ireceptor_patwo_metadata_list.append(repertoire_dict)
ireceptor_patwo_metadata_table = pd.DataFrame(ireceptor_patwo_metadata_list).drop_duplicates()
ireceptor_patwo_metadata_table['repertoire_id'] = ireceptor_patwo_metadata_table['repertoire_id'].astype('str')
ireceptor_patwo_metadata_table = pd.merge(ireceptor_patwo_metadata_table, ireceptor_patwo_trb_table, on = "repertoire_id", how = "inner")
ireceptor_patwo_metadata_table

### iReceptor PA2 sequence table

In [ ]:
ireceptor_patwo_sequence_table = ireceptor_patwo_metadata_table[['junction_aa']]
ireceptor_patwo_sequence_table = ireceptor_patwo_sequence_table.loc[ireceptor_patwo_sequence_table['junction_aa'].notna()]
ireceptor_patwo_sequence_table = ireceptor_patwo_sequence_table.loc[~ireceptor_patwo_sequence_table['junction_aa'].str.contains('_')]
ireceptor_patwo_sequence_table = ireceptor_patwo_sequence_table.rename(columns={'junction_aa': 'trb'})
ireceptor_patwo_sequence_table['sequence'] = ireceptor_patwo_sequence_table['trb'] + ';'
ireceptor_patwo_sequence_table = ireceptor_patwo_sequence_table.drop_duplicates()
ireceptor_patwo_sequence_table

## iReceptor public archive part 3

The third fragment of the iReceptor public archive contains 118771 paired TCR sequences (from https://ipa6.ireceptor.org/airr/v1/)

In [ ]:
ireceptor_pathree_path = Path('data/tcr_llm/databases/ireceptor/ireceptor-public-archive_part3.tsv')
ireceptor_pathree_table = pd.read_csv(ireceptor_pathree_path, sep = "\t")
ireceptor_pathree_table = ireceptor_pathree_table[['data_processing_id', 'repertoire_id', 
    'cell_id', 'clone_id', 'productive', 'locus', 'v_call', 'd_call', 'j_call',
    'cdr1_aa', 'cdr2_aa', 'cdr3_aa', 'junction_aa']]
ireceptor_pathree_table = ireceptor_pathree_table.loc[ireceptor_pathree_table['productive'] == 'T']
ireceptor_pathree_tra_table = ireceptor_pathree_table.loc[ireceptor_pathree_table['locus'] == 'TRA'][['data_processing_id', 'cell_id', 'clone_id', 'v_call', 'd_call', 'j_call', 'junction_aa']]
ireceptor_pathree_tra_table = ireceptor_pathree_tra_table.rename(columns={'v_call': 'trav_gene',
    'd_call': 'trad_gene', 'j_call': 'traj_gene', 'junction_aa': 'tra_junction_aa'})
ireceptor_pathree_trb_table = ireceptor_pathree_table.loc[ireceptor_pathree_table['locus'] == 'TRB'][['data_processing_id', 'cell_id', 'clone_id', 'v_call', 'd_call', 'j_call', 'junction_aa']]
ireceptor_pathree_trb_table = ireceptor_pathree_trb_table.rename(columns={'v_call': 'trbv_gene',
    'd_call': 'trbd_gene', 'j_call': 'trbj_gene', 'junction_aa': 'trb_junction_aa'})
ireceptor_pathree_tcr_table = pd.merge(ireceptor_pathree_tra_table, ireceptor_pathree_trb_table, 
    on=['data_processing_id', 'cell_id', 'clone_id']).rename(columns={'data_processing_id': 'repertoire_id'})
ireceptor_pathree_tcr_table = ireceptor_pathree_tcr_table.drop(columns=['cell_id', 'clone_id'])
ireceptor_pathree_tcr_table

### iReceptor PA3 metadata

In [ ]:
ireceptor_pathree_meta_path = Path("data/tcr_llm/databases/ireceptor/ireceptor-public-archive_part3-metadata.json")
with open(ireceptor_pathree_meta_path) as meta_file:
    ireceptor_pathree_metadata = json.load(meta_file)
ireceptor_pathree_metadata_list = list()
for repertoires in ireceptor_pathree_metadata["Repertoire"]:
    repertoire_dict = dict()
    repertoire_dict['study_id'] = repertoires['study']['study_id']
    repertoire_dict['repertoire_id'] = repertoires['data_processing'][0]['data_processing_id']
    repertoire_dict['host_organism'] = repertoires['subject']['species']['label']
    if repertoires['subject']['diagnosis'][0]['disease_diagnosis']['label'] == None:
        repertoire_dict['condition_studies'] = repertoires['subject']['diagnosis'][0]['study_group_description']
    else:
        repertoire_dict['condition_studies'] = repertoires['subject']['diagnosis'][0]['study_group_description'] + ' ' + repertoires['subject']['diagnosis'][0]['disease_diagnosis']['label']
    repertoire_dict['age'] = repertoires['subject']['age']
    repertoire_dict['sex'] = repertoires['subject']['sex']
    repertoire_dict['population_surveyed'] = repertoires['subject']['race']
    mhc_list = set()
    for mhc_genotypes in repertoires['subject']['genotype']['mhc_genotype_set']['mhc_genotype_list']:
        for alleles in mhc_genotypes['mhc_alleles']:
            if alleles['allele_designation'] == None:
                continue
            else:
                mhc_list.add(alleles['allele_designation'])
    repertoire_dict['mhc_profile'] = ','.join(sorted(list(mhc_list))) if list(mhc_list) else pd.NA
    repertoire_dict['source_tissue'] = repertoires['sample'][0]['tissue']['label']
    ireceptor_pathree_metadata_list.append(repertoire_dict)
ireceptor_pathree_metadata_table = pd.DataFrame(ireceptor_pathree_metadata_list).drop_duplicates()
ireceptor_pathree_metadata_table = pd.merge(ireceptor_pathree_metadata_table,
                                            ireceptor_pathree_tcr_table,
                                            on = "repertoire_id", how = "inner")
ireceptor_pathree_metadata_table

### iReceptor PA3 sequence table

In [ ]:
ireceptor_pathree_sequence_table = ireceptor_pathree_metadata_table[['tra_junction_aa', 'trb_junction_aa']].rename(columns={'tra_junction_aa': 'tra', 'trb_junction_aa': 'trb'})
ireceptor_pathree_sequence_table['sequence'] = ireceptor_pathree_sequence_table['tra'] + ' ' + ireceptor_pathree_sequence_table['trb'] + ';'
ireceptor_pathree_sequence_table = ireceptor_pathree_sequence_table.drop_duplicates()
ireceptor_pathree_sequence_table

## University of munster
The university of munster dataset contains 184166 TRB sequences (from https://agschwab.uni-muenster.de/airr/v1/) from bulk TCR sequencing datasets

In [ ]:
umunster_path = Path('/home/sravisha/projects/quest/data/tcr_llm/databases/ireceptor/university-of-munster.tsv')
umunster_table = pd.read_csv(umunster_path, sep = "\t", nrows=500000)
umunster_table = umunster_table[['repertoire_id', 
    'cell_id', 'clone_id', 'productive', 'locus', 'v_call', 'd_call', 'j_call',
    'cdr1_aa', 'cdr2_aa', 'cdr3_aa', 'junction_aa']]
umunster_table = umunster_table.loc[umunster_table['productive'] == 'T']
umunster_trb_table = umunster_table.loc[umunster_table['locus'] == 'TRB'][['repertoire_id',  'cell_id', 'clone_id', 'v_call', 'd_call', 'j_call', 'junction_aa']]
umunster_trb_table['repertoire_id'] = umunster_trb_table['repertoire_id'].astype('str')
umunster_trb_table

### University of munster metadata

In [ ]:
umunster_meta_path = Path("/home/sravisha/projects/quest/data/tcr_llm/databases/ireceptor/university-of-munster-metadata.json")
with open(umunster_meta_path) as meta_file:
    umunster_meta_path = json.load(meta_file)
umunster_metadata_list = list()
for repertoires in umunster_meta_path["Repertoire"]:
    repertoire_dict = dict()
    repertoire_dict['study_id'] = repertoires['study']['study_id']
    repertoire_dict['repertoire_id'] = repertoires['repertoire_id']
    repertoire_dict['host_organism'] = repertoires['subject']['species']['label']
    if repertoires['subject']['diagnosis'][0]['disease_diagnosis']['label'] == None:
        repertoire_dict['condition_studies'] = repertoires['subject']['diagnosis'][0]['study_group_description']
    else:
        repertoire_dict['condition_studies'] = repertoires['subject']['diagnosis'][0]['study_group_description'] + ' ' + repertoires['subject']['diagnosis'][0]['disease_diagnosis']['label']
    repertoire_dict['age'] = repertoires['subject']['age']
    repertoire_dict['sex'] = repertoires['subject']['sex']
    repertoire_dict['population_surveyed'] = repertoires['subject']['race']
    mhc_list = set()
    if 'genotype' in repertoires['subject']:
        for mhc_genotypes in repertoires['subject']['genotype']['mhc_genotype_set']['mhc_genotype_list']:
            for alleles in mhc_genotypes['mhc_alleles']:
                if alleles['allele_designation'] == None:
                    continue
                else:
                    mhc_list.add(alleles['allele_designation'])
    repertoire_dict['mhc_profile'] = ','.join(sorted(list(mhc_list))) if list(mhc_list) else pd.NA
    repertoire_dict['source_tissue'] = repertoires['sample'][0]['tissue']['label']
    umunster_metadata_list.append(repertoire_dict)
umunster_meta_path = pd.DataFrame(umunster_metadata_list).drop_duplicates()
umunster_meta_path['repertoire_id'] = umunster_meta_path['repertoire_id'].astype('str')
umunster_meta_path = pd.merge(umunster_meta_path, umunster_table, on = "repertoire_id", how = "inner")
umunster_meta_path

### University of munster sequence table

In [ ]:
umunster_sequence_table = umunster_meta_path[['junction_aa']]
umunster_sequence_table = umunster_sequence_table.loc[umunster_sequence_table['junction_aa'].notna()]
umunster_sequence_table = umunster_sequence_table.loc[~umunster_sequence_table['junction_aa'].str.contains('_')]
umunster_sequence_table = umunster_sequence_table.rename(columns={'junction_aa': 'trb'})
umunster_sequence_table['sequence'] = umunster_sequence_table['trb'] + ';'
umunster_sequence_table = umunster_sequence_table.drop_duplicates()
umunster_sequence_table

## VDJServer

VDJServer contains bulk TCR sequencing data of both the alpha and beta chain sequencing datasets. This database contains a total of  193647 sequences (from https://vdjserver.org/airr/v1/)

In [ ]:
vdjserver_path = Path('/home/sravisha/projects/quest/data/tcr_llm/databases/ireceptor/vdjserver.tsv')
vdjserver_table = pd.read_csv(vdjserver_path, sep = "\t")
vdjserver_table = vdjserver_table[['data_processing_id', 'repertoire_id', 
    'cell_id', 'clone_id', 'productive', 'locus', 'v_call', 'd_call', 'j_call',
    'cdr1_aa', 'cdr2_aa', 'cdr3_aa', 'junction_aa']]
vdjserver_table = vdjserver_table.loc[vdjserver_table['productive']]
vdjserver_table_tra_table = vdjserver_table.loc[vdjserver_table['locus'] == 'TRA'][['repertoire_id', 'cell_id', 'clone_id', 'v_call', 'd_call', 'j_call', 'junction_aa']]
vdjserver_table_tra_table = vdjserver_table_tra_table.rename(columns={'v_call': 'trav_gene',
    'd_call': 'trad_gene', 'j_call': 'traj_gene', 'junction_aa': 'tra_junction_aa'})
vdjserver_table_trb_table = vdjserver_table.loc[vdjserver_table['locus'] == 'TRB'][['repertoire_id', 'cell_id', 'clone_id', 'v_call', 'd_call', 'j_call', 'junction_aa']]
vdjserver_table_trb_table = vdjserver_table_trb_table.rename(columns={'v_call': 'trbv_gene',
    'd_call': 'trbd_gene', 'j_call': 'trbj_gene', 'junction_aa': 'trb_junction_aa'})
vdjserver_table_tcr_table = pd.merge(vdjserver_table_tra_table, vdjserver_table_trb_table, 
    on=['repertoire_id', 'cell_id', 'clone_id']) #.rename(columns={'data_processing_id': 'repertoire_id'})
vdjserver_table_tcr_table = vdjserver_table_tcr_table.drop(columns=['cell_id', 'clone_id'])
vdjserver_table_tcr_table

### VDJServer metadata

In [ ]:
vdjserver_meta_path = Path("/home/sravisha/projects/quest/data/tcr_llm/databases/ireceptor/vdjserver-metadata.json")
with open(vdjserver_meta_path) as meta_file:
    vdjserver_metadata = json.load(meta_file)
vdjserver_metadata_list = list()
for repertoires in vdjserver_metadata["Repertoire"]:
    repertoire_dict = dict()
    repertoire_dict['study_id'] = repertoires['study']['study_id']
    repertoire_dict['repertoire_id'] = repertoires['repertoire_id']
    repertoire_dict['host_organism'] = repertoires['subject']['species']['label']
    if repertoires['subject']['diagnosis'][0]['disease_diagnosis']['label'] == None:
        repertoire_dict['condition_studies'] = repertoires['subject']['diagnosis'][0]['study_group_description']
    else:
        repertoire_dict['condition_studies'] = repertoires['subject']['diagnosis'][0]['study_group_description'] + ' ' + repertoires['subject']['diagnosis'][0]['disease_diagnosis']['label']
    repertoire_dict['age'] = repertoires['subject']['age'] if 'age' in repertoires['subject'] else pd.NA
    repertoire_dict['sex'] = repertoires['subject']['sex']
    repertoire_dict['population_surveyed'] = repertoires['subject']['race']
    mhc_list = set()
    for mhc_genotypes in repertoires['subject']['genotype']['mhc_genotype_set']['mhc_genotype_list']:
        for alleles in mhc_genotypes['mhc_alleles']:
            if alleles['allele_designation'] == None:
                continue
            else:
                mhc_list.add(alleles['allele_designation'])
    repertoire_dict['mhc_profile'] = ','.join(sorted(list(mhc_list))) if list(mhc_list) else pd.NA
    repertoire_dict['source_tissue'] = repertoires['sample'][0]['tissue']['label']
    vdjserver_metadata_list.append(repertoire_dict)
vdjserver_metadata_table = pd.DataFrame(vdjserver_metadata_list).drop_duplicates()
vdjserver_metadata_table = pd.merge(vdjserver_metadata_table,
                                            vdjserver_table_tcr_table,
                                            on = "repertoire_id", how = "inner")
vdjserver_metadata_table

### VDJServer sequencing table

In [ ]:
vdjserver_sequence_table = vdjserver_metadata_table[['tra_junction_aa', 'trb_junction_aa']].rename(columns={'tra_junction_aa': 'tra', 'trb_junction_aa': 'trb'})
vdjserver_sequence_table['sequence'] = vdjserver_sequence_table['tra'] + ' ' + vdjserver_sequence_table['trb'] + ';'
vdjserver_sequence_table = vdjserver_sequence_table.drop_duplicates()
vdjserver_sequence_table

# McPAS-TCR

In [ ]:
mcpastcr_path = Path("data/tcr_llm/databases/McPASDB/McPAS-TCR.csv")
mcpastcr_table = pd.read_csv(mcpastcr_path)

## McPAS-TCR metadata

In [ ]:
# Make a copy of mcpastcr_table for renaming and updating columns
mcpastcr_metadata_table = mcpastcr_table

# Rename specific columns to standardized metadata names
mcpastcr_metadata_table = mcpastcr_metadata_table.rename(columns={
    'TRAV': 'trav_gene',                   # TCR alpha variable gene
    'TRAJ': 'traj_gene',                   # TCR alpha joining gene
    'CDR3.alpha.aa': 'tra_junction_aa',    # TCR alpha CDR3 region (junction) amino acid sequence
    'TRBV': 'trbv_gene',                   # TCR beta variable gene
    'TRBD': 'trbd_gene',                   # TCR beta diversity gene
    'TRBJ': 'trbj_gene',                   # TCR beta joining gene
    'CDR3.beta.aa': 'trb_junction_aa',     # TCR beta CDR3 region (junction) amino acid sequence
    'PubMed.ID': 'study_id',               # Identifier for the publication
    'Category': 'host_condition',          # Health condition of the host
    'Species': 'host_organism',            # Organism species of the host
    'Epitope.peptide': 'peptide',          # Epitope peptide sequence
    'Antigen.identification.method': 'assay_method',  # Method used for antigen identification
    'Antigen.protein': 'epitope_source_molecule',     # Source protein of the epitope
    'Protein.ID': 'epitope_reference_name',           # Reference name for the source protein
    'Pathology': 'epitope_source_organism',           # Organism from which epitope is derived
    'MHC': 'mhc_restriction'                          # MHC restriction for the TCR
})

# Add new columns with metadata that may be missing in the dataset
mcpastcr_metadata_table['data_source'] = 'McPAS-TCR'         # Source of the dataset
mcpastcr_metadata_table['study_id_type'] = 'PMID'            # Type of study identifier (PubMed ID)

# Reorder columns to a standardized format for consistency
mcpastcr_metadata_table = mcpastcr_metadata_table[
    ['data_source', 'study_id', 'study_id_type', 'repertoire_id', 
     'host_organism', 'trav_gene', 'traj_gene', 'tra_junction_aa',  
     'trbv_gene', 'trbd_gene', 'trbj_gene', 'trb_junction_aa', 
     'peptide', 'epitope_reference_name', 'epitope_source_molecule', 
     'epitope_source_organism', 'mhc_restriction', 'assay_method']
]

# Output the transformed metadata table
mcpastcr_metadata_table



## McPAS-TCR sequence table

In [ ]:
# Select relevant columns from the metadata table for the sequence table
mcpastcr_sequence_table = mcpastcr_metadata_table[['tra_junction_aa', 'trb_junction_aa', 'peptide', 'mhc_restriction']]

# Apply the `transform_mhc_restriction` function to the MHC restriction column, using `combined_fasta_dict` for mapping
mcpastcr_sequence_table.loc[:, 'mhc_restriction'] = mcpastcr_sequence_table['mhc_restriction'].apply(
    transform_mhc_restriction, fasta_dict=combined_fasta_dict
)

# Add a new 'mhc' column by mapping each transformed MHC restriction value to `combined_fasta_dict`
mcpastcr_sequence_table.loc[:, 'mhc'] = mcpastcr_sequence_table['mhc_restriction'].apply(
    lambda mhc: combined_fasta_dict.get(mhc, pd.NA)
)

# Split the 'peptide' column on '/' (if it contains multiple peptides), expand to separate rows for each peptide
mcpastcr_sequence_table = mcpastcr_sequence_table.assign(
    values=mcpastcr_sequence_table['peptide'].str.split('/')
).explode('peptide', ignore_index=True)

# Rename the TCR alpha and beta junction columns to 'tra' and 'trb' for consistency
mcpastcr_sequence_table = mcpastcr_sequence_table.rename(
    columns={'tra_junction_aa': 'tra', 'trb_junction_aa': 'trb'}
)

# Drop the original 'mhc_restriction' column, since it's no longer needed
mcpastcr_sequence_table = mcpastcr_sequence_table.drop(columns=['mhc_restriction'])

# Create a new 'sequence' column by concatenating 'tra', 'trb', 'peptide', and 'mhc' values into a single string
# Join non-null values with a space, and exclude values containing '*'
mcpastcr_sequence_table['sequence'] = mcpastcr_sequence_table[['tra', 'trb', 'peptide', 'mhc']].agg(
    lambda x: ' '.join([val for val in x if pd.notna(val) and '*' not in val]) + ';',
    axis=1
)

# Select relevant columns and remove duplicate rows
mcpastcr_sequence_table = mcpastcr_sequence_table[['tra', 'trb', 'peptide', 'mhc', 'sequence']]
mcpastcr_sequence_table.drop_duplicates()


# TCRdb

In [ ]:
# Define the path to the TCRdb directory and initialize an empty list to store dataframes
tcrdb_path = Path("data/tcr_llm/databases/TCRdb")
tcrdb_file_list = list()

# Iterate through all .tsv files in the TCRdb directory and its subdirectories
for tcrdb_files in tcrdb_path.rglob("*.tsv"):
    study_id = tcrdb_files.stem  # Use the filename (without extension) as the study_id
    tcr_table = pd.read_csv(tcrdb_files, sep="\t")  # Read the .tsv file into a dataframe
    
    # Rename columns to standardize naming across data sources
    tcr_table.rename(columns={
        'RunId': 'repertoire_id',       # Unique identifier for each TCR repertoire
        'Vregion': 'trbv_gene',         # TCR beta variable gene
        'Dregion': 'trbd_gene',         # TCR beta diversity gene
        'Jregion': 'trbj_gene',         # TCR beta joining gene
        'AASeq': 'trb_junction_aa'      # TCR beta CDR3 region (junction) amino acid sequence
    }, inplace=True)
    
    # Remove 'cloneFraction' column as it is not needed for this analysis
    tcr_table.drop(columns=['cloneFraction'], inplace=True)
    
    # Add the study ID as a new column for tracking the source of each row
    tcr_table['study_id'] = study_id
    
    # Replace any 'Unknown' values with NA to handle missing data consistently
    tcr_table = tcr_table.replace('Unknown', pd.NA)
    
    # Reorder columns for consistency and select only relevant columns
    tcr_table = tcr_table[['study_id', 'repertoire_id', 'trbv_gene', 'trbd_gene', 
                           'trbj_gene', 'trb_junction_aa']]
    
    # Append the processed dataframe to the list
    tcrdb_file_list.append(tcr_table)

# Concatenate all dataframes from the list into a single dataframe
tcrdb_metadata_table = pd.concat(tcrdb_file_list)

# Create the sequence table by selecting and renaming columns as needed
tcrdb_sequence_table = tcrdb_metadata_table[['trb_junction_aa']].rename(columns={'trb_junction_aa': 'trb'})

# Create a 'sequence' column by appending a semicolon to each 'trb' value
tcrdb_sequence_table['sequence'] = tcrdb_sequence_table['trb'] + ';'

# Remove duplicate sequences to get unique rows
tcrdb_sequence_table = tcrdb_sequence_table.drop_duplicates()

# Display the resulting sequence table
tcrdb_sequence_table



# VdjDB

In [ ]:
# Define the path to the VDJdb full data file
vdjdb_path = Path("data/tcr_llm/databases/VdjDB/vdjdb_full.txt")

# Read the VDJdb data into a DataFrame, specifying tab as the separator
vdjdb_table = pd.read_csv(vdjdb_path, sep="\t")

# Filter the data to include only human (Homo sapiens) entries
vdjdb_table = vdjdb_table.loc[vdjdb_table['species'] == 'HomoSapiens']

# Rename columns to standardize naming across datasets and improve clarity
vdjdb_table = vdjdb_table.rename(columns={
    'cdr3.alpha': 'tra_junction_aa',                # TCR alpha CDR3 region (junction) amino acid sequence
    'v.alpha': 'trav_gene',                         # TCR alpha variable gene
    'j.alpha': 'traj_gene',                         # TCR alpha joining gene
    'cdr3.beta': 'trb_junction_aa',                 # TCR beta CDR3 region (junction) amino acid sequence
    'v.beta': 'trbv_gene',                          # TCR beta variable gene
    'd.beta': 'trbd_gene',                          # TCR beta diversity gene
    'j.beta': 'trbj_gene',                          # TCR beta joining gene
    'species': 'host_organism',                     # Species of the host organism
    'mhc.a': 'mhc_restriction',                     # MHC restriction (type A)
    'mhc.b': 'mhc_restriction_two',                 # MHC restriction (type B)
    'mhc.class': 'mhc_class',                       # MHC class (I or II)
    'antigen.epitope': 'peptide',                   # Epitope peptide sequence
    'antigen.gene': 'epitope_source_molecule',      # Source molecule for the epitope
    'antigen.species': 'epitope_source_organism',   # Organism from which the epitope is derived
    'reference.id': 'study_id',                     # Reference ID for the study
    'method.verification': 'assay_method',          # Method used for verification
    'meta.epitope.id': 'epitope_reference_name',    # ID of the epitope reference
    'meta.tissue': 'source_tissue',                 # Tissue source of the sample
    'meta.donor.MHC': 'mhc_profile'                # MHC profile of the donor
})

# Drop columns that are not needed for further analysis
vdjdb_metadata_table = vdjdb_table.drop(columns=[
    'method.identification',      # Identification method (not needed here)
    'method.frequency',           # Frequency method
    'method.singlecell',          # Single-cell method
    'method.sequencing',          # Sequencing method
    'meta.study.id',              # Study ID metadata
    'meta.cell.subset',           # Cell subset information
    'meta.subject.cohort',        # Subject cohort information
    'meta.subject.id',            # Subject ID
    'meta.replica.id',            # Replica ID
    'meta.clone.id',              # Clone ID
    'meta.donor.MHC.method',      # Donor MHC method
    'meta.structure.id',          # Structure ID
    'cdr3fix.alpha',              # Fixed alpha CDR3 sequence
    'cdr3fix.beta',               # Fixed beta CDR3 sequence
    'vdjdb.score'                 # VDJdb score (not needed here)
])

# Display the resulting metadata table
vdjdb_metadata_table


In [ ]:
# Select relevant columns for the sequence table
vdjdb_sequence_table = vdjdb_metadata_table[['tra_junction_aa', 'trb_junction_aa', 'peptide', 'mhc_restriction', 'mhc_restriction_two']]

# Apply the MHC transformation function to both 'mhc_restriction' and 'mhc_restriction_two' columns
vdjdb_sequence_table.loc[:, 'mhc_restriction'] = vdjdb_sequence_table['mhc_restriction'].apply(
    transform_mhc_restriction, fasta_dict=combined_fasta_dict
)
vdjdb_sequence_table.loc[:, 'mhc_restriction_two'] = vdjdb_sequence_table['mhc_restriction_two'].apply(
    transform_mhc_restriction, fasta_dict=combined_fasta_dict
)

# Map the transformed MHC restrictions to the corresponding MHC sequences from 'combined_fasta_dict'
vdjdb_sequence_table['mhc_one'] = vdjdb_sequence_table['mhc_restriction'].apply(lambda mhc: combined_fasta_dict.get(mhc, pd.NA))
vdjdb_sequence_table['mhc_two'] = vdjdb_sequence_table['mhc_restriction_two'].apply(lambda mhc: combined_fasta_dict.get(mhc, pd.NA))

# Drop the original MHC restriction columns, keeping only the mapped sequences
vdjdb_sequence_table = vdjdb_sequence_table.drop(columns=['mhc_restriction', 'mhc_restriction_two'])

# Rename columns for consistency and clarity
vdjdb_sequence_table = vdjdb_sequence_table.rename(columns={
    'tra_junction_aa': 'tra',        # TCR alpha junction amino acid sequence
    'trb_junction_aa': 'trb'         # TCR beta junction amino acid sequence
})

# Concatenate non-empty values from the selected columns into a single 'sequence' field, with ';' as a suffix
vdjdb_sequence_table['sequence'] = vdjdb_sequence_table[['tra', 'trb', 'peptide', 'mhc_one', 'mhc_two']].agg(
    lambda x: ' '.join([val for val in x if pd.notna(val) and '*' not in val]) + ';',
    axis=1
)

# Remove duplicate rows to ensure unique entries in the sequence table
vdjdb_sequence_table = vdjdb_sequence_table.drop_duplicates()
